In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH14-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH14-Retriever


# ContextualCompressionRetriever

1. 개요
    - 사용자 질의에 맞춰 검색된 Context를 의미적으로 압축 
    - LLM에 전달할 문맥을 더 정제된 형태로 최적화 

2. 주요 특징 
    - 문맥 정제 : 검색된 문서에서 LLM에게 불필요한 내용 제거, 핵심 정보만 추출   
    - 의미 기반 압축 : 단순 토큰 절단이 아닌, 의미 파악 후 핵심만 요약 
    - 노이즈 제거 : 관련 없는 정보, 중복, 부정확한 부분을 사전에 제거 가능 

[Reference] https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.contextual_compression.ContextualCompressionRetriever.html

In [3]:
# 문서를 예쁘게 출력하기 위한 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"문서 {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## Basic Retriever 설정 

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [5]:
# 문서 로딩 
loader = TextLoader("./data/appendix-keywords.txt")

# 청크 분할
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = loader.load_and_split(text_splitter)

# 검색기 정의 
# as_retriever : default 4개 문서 검색
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

In [6]:
# 문서 검색
docs = retriever.invoke("Semantic Search 에 대해서 알려줘.")

# 결과 출력 
pretty_print_docs(docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------
문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
----------------------------------------------------------------------------------------------------
문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
----------------------------------------------------------------------------------------------------
문서 4:

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이

## 맥락 압축 

In [7]:
from langchain_teddynote.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain_openai import ChatOpenAI

In [8]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")  

# LLM을 사용하여 문서 압축기 생성
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    # 문서 압축기와 검색기를 사용하여 컨텍스트 압축 리트리버 생성
    base_compressor=compressor,
    base_retriever=retriever,
)

In [9]:
pretty_print_docs(retriever.invoke("Semantic Search 에 대해서 알려줘."))

print("=========================================================")
print("============== LLMChainExtractor 적용 후 ==================")

compressed_docs = (compression_retriever.invoke("Semantic Search 에 대해서 알려줘."))
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------
문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
----------------------------------------------------------------------------------------------------
문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
----------------------------------------------------------------------------------------------------
문서 4:

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이

## 문서 필터링 
### LLMChainFilter 
- 초기 검색된 문서 중 어떤 문서를 필터링 할지 결정하기 위함 
- 해당 필터는 문서 내용을 압축(변경)하지 않고, 문서를 선택적으로 반환 

In [10]:
from langchain_teddynote.document_compressors import LLMChainFilter

In [11]:
_filter = LLMChainFilter.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter,
    base_retriever=retriever,
)

In [12]:
compressed_docs = compression_retriever.invoke("Semantic Search 에 대해서 알려줘.")
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


<- 해당 실행 결과를 LangSmith에서 확인하기 



### EmbeddingsFilter 

- 각각의 검색된 문서에 대해 추가적인 LLM 호출을 수행하는 것은 비용이 많이 들고 속도 느림 
- 해당 필터는 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환 -> 계산 비용과 시간 절약 

In [13]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_openai import OpenAIEmbeddings

In [14]:
embeddings = OpenAIEmbeddings()

# 유사도 임계값이 0.86인 EmbeddingsFilter 객체 생성
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.86)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, 
    base_retriever=retriever,
)

In [15]:
compressed_docs = compression_retriever.invoke("Semantic Search 에 대해서 알려줘.")
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


<- 해당 실행 결과를 LangSmith에서 확인하기 (위 결과와 비교도 해보기)

## Piepeline 

In [16]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

In [17]:
# 청크 분할 
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 중복 필터 생성
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

# 관련성 필터를 생성
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.86)

# 문서 압축 파이프라인 
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[
        splitter,
        redundant_filter,
        relevant_filter,
        LLMChainExtractor.from_llm(llm),
    ]
)

# Pipeline
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor,
    base_retriever=retriever,
)

In [18]:
compressed_docs = compression_retriever.invoke("Semantic Search 에 대해서 알려줘.")
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.


<- 해당 실행 결과를 LangSmith에서 확인하기 (Tokens, Latency, Tracing)

-----
** End of Documents **